In [1]:
import pandas as pd
from tqdm import tqdm
from database.strategy import Strategy
from database.market import Market
from transformer.column_transformer import ColumnTransformer
from transformer.date_transformer import DateTransformer
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import matplotlib.pyplot as plt 
from datetime import datetime, timedelta, timezone
import math
import numpy as np

In [2]:
strat_db = Strategy("unity")
market = Market()

In [3]:
market.connect()
sp5 = market.retrieve_data("sp500")
market.close()

In [4]:
start = datetime(2018,1,1)
end = datetime(2021,1,1)
reload = False
datasets = [
            "pdr",
#             "tiingo",
#             "finnhub"
]
dataset = "pdr"

In [5]:
sim = []
if reload:
    for ticker in tqdm(sp5["Symbol"]):
        try:
            market.connect()
            prices = market.retrieve_price_data("{}_prices".format(dataset),ticker)
            market.close()
            if dataset == "pdr":
                prices = ColumnTransformer.rename_columns(prices," ")
            else:
                prices = ColumnTransformer.rename_columns(prices,"")
            prices = DateTransformer.convert_to_date(dataset,prices,"date")
            prices["year"] = [x.year for x in prices["date"]]
            prices["week"] = [x.week for x in prices["date"]]
            prices["quarter"] = [x.quarter for x in prices["date"]]
            prices["rolling100"] = prices["adjclose"].rolling(window=100).mean()
            prices["rolling7"] = prices["adjclose"].rolling(window=7).mean()
            current_quarterly_max = []
            for row in prices.iterrows():
                current_date  = row[1]["date"]
                current_quarter = row[1]["quarter"]
                current_year = row[1]["year"]
                relevant = prices[(prices["year"] == current_year) & (prices["quarter"]==current_quarter) & (prices["date"] <= current_date)]
                current_quarterly_max.append(relevant["adjclose"].max())
            prices["cqm"] = current_quarterly_max
            prices["d1"] = [1 if x > 0 else 0 for x in prices["adjclose"].diff()]
            prices["d2"] = [1 if x > 0 else 0 for x in prices["d1"].diff()]
            prices["100d"] = (prices["rolling100"] - prices["adjclose"]) / prices["adjclose"]
            prices["7d"] = (prices["rolling7"] - prices["adjclose"]) / prices["adjclose"]
            prices["qd"] = (prices["cqm"] - prices["adjclose"]) / prices["adjclose"]
            sim.append(prices)
        except Exception as e:
            print(ticker, str(e))
            continue

In [19]:
strat_db.connect()
sim = strat_db.retrieve_data("headless_sim")
strat_db.close()

In [12]:
for col in ["100d","7d","qd"]:
    sim[col] = sim[col] * -1


In [16]:
todays_sim = sim[(sim["100d"]>0) & (sim["7d"]<0)] 

In [28]:
sim[sim["7d"] >= 0.03]

,_id,date,high,low,open,close,volume,adjclose,ticker,year,week,quarter,rolling100,rolling7,cqm,d1,d2,100d,7d,qd
23,6061fb994ff5696e1a0f976e,2008-06-26,72.300003,70.239998,72.290001,70.260002,7371800.0,49.418934,MMM,2008,26,2,54.570506,51.211542,57.933537,0,0,0.104243,0.036274,0.172294
24,6061fb994ff5696e1a0f976f,2008-06-27,70.480003,68.889999,70.480003,69.510002,7481700.0,48.891422,MMM,2008,26,2,54.510171,50.730236,57.933537,0,0,0.114923,0.037610,0.184943
88,6061fb994ff5696e1a0f97af,2008-09-29,68.930000,65.510002,68.930000,66.320000,8246900.0,46.977039,MMM,2008,40,3,50.929414,49.224501,52.316849,0,0,0.084134,0.047842,0.113668
91,6061fb994ff5696e1a0f97b2,2008-10-02,67.750000,65.510002,66.910004,65.599998,7880100.0,46.467037,MMM,2008,40,4,50.741407,48.103299,47.734959,0,0,0.091987,0.035213,0.027286
92,6061fb994ff5696e1a0f97b3,2008-10-03,67.250000,64.510002,66.089996,64.599998,6588000.0,45.758694,MMM,2008,40,4,50.659632,47.701568,47.734959,0,0,0.107104,0.042459,0.043189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545193,6061ef6a4ff5696e1ae823ab,2021-03-23,11.930000,10.370000,11.460000,10.660000,87923200.0,10.660000,AMC,2021,12,1,5.472000,13.100000,19.900000,0,0,-0.486679,0.228893,0.866792
1545194,6061ef6a4ff5696e1ae823ac,2021-03-24,11.210000,8.930000,10.820000,9.020000,81850700.0,9.020000,AMC,2021,12,1,5.536100,12.382857,19.900000,0,0,-0.386242,0.372822,1.206208
1545195,6061ef6a4ff5696e1ae823ad,2021-03-25,11.320000,8.950000,8.960000,10.940000,131192800.0,10.940000,AMC,2021,12,1,5.620300,12.085714,19.900000,1,1,-0.486261,0.104727,0.819013
1545196,6061ef6a4ff5696e1ae823ae,2021-03-26,11.530000,10.010000,11.270000,10.240000,83778100.0,10.240000,AMC,2021,12,1,5.699100,11.611429,19.900000,0,0,-0.443447,0.133929,0.943359


In [29]:
def backtest(sim,industries,delta_column,value,hpr,delta_req):
        trades = []
        blacklist = pd.DataFrame([{"ticker":"ZZZZZ","start":datetime(2016,4,1),"end":datetime(2016,4,14)}])
        if not value:
            print("not value")
            for col in ["100d","7d","qd"]:
                sim[col] = sim[col] * -1
        for industry in tqdm(industries):
            date = start
            industry_tickers = list(sp5[sp5["GICS Sector"] == industry]["Symbol"])
            while date <= end:
                if date >= end:
                    break
                if date.weekday() > 4 \
                        or date == datetime(2020,2,20):
                    date = date + timedelta(days=7-date.weekday())
                blacklist_tickers = blacklist[(blacklist["start"] <= date) & (blacklist["end"] >= date)]["ticker"]
                if delta_column == "7d":
                    delta_req = float(delta_req/3)
                todays_sim = sim[(~sim["ticker"].isin(blacklist_tickers)) & (sim["ticker"].isin(industry_tickers)) &
                                          (sim["date"] == date) & (sim["100d"]>0) & (sim["7d"]>0)] 
                if todays_sim.index.size >= 1:
                    offerings = todays_sim.sort_values(delta_column,ascending=False)
                    for offering in range(offerings.index.size):
                        try:
                            trade_ticker = offerings.iloc[offering]["ticker"]
                            try:
                                trade = sim[(sim["ticker"] == trade_ticker) & (sim["date"] > date)].iloc[0]
                                sell_date = trade["date"] + timedelta(days=1)
                                sell_trades = sim[(sim["date"] >= sell_date) & (sim["ticker"] == trade["ticker"])]
                                ## if there aren't any listed prices left for the ticker
                                if sell_trades.index.size < 1:
                                    ## if there aren't any more ticker's left to vet
                                    if offering == offerings.index.size - 1:
                                        date = sell_date + timedelta(days=1)
                                        print(date,"no more stock vets")
                                        break
                                    else:
                                        print(date,"stock had no more listed prices")
                                        continue
                                else:
                                    if date > datetime(2020,10,1):
                                        max_hpr = int((end - date).days) - 1
                                    else:
                                        max_hpr = hpr
                                    sell_trades["gain"] = (sell_trades["adjclose"] - trade["adjclose"]) / trade["adjclose"]
                                    sell_trades["hpr"] = [(x - trade["date"]).days for x in sell_trades["date"]]
                                    hpr_sell_trades = sell_trades[sell_trades["hpr"] < max_hpr]
                                    hpr_sell_trades["hit"] = hpr_sell_trades["gain"] >= delta_req
                                    delta_hit = hpr_sell_trades[(hpr_sell_trades["hit"] == True)]
                                    ## if we never hit the mark
                                    if delta_hit.index.size < 1:
                                        sell_trade = sell_trades[(sell_trades["hpr"] >= max_hpr)].sort_values("hpr",ascending=True).iloc[0]
                                    else:
                                        sell_trade = delta_hit.iloc[0]
                                    trade["sell_price"] = sell_trade["adjclose"]
                                    trade["delta_req"] = delta_req
                                    trade["sell_date"] = sell_trade["date"]
                                    trade["sell_delta"] = (sell_trade["adjclose"] - trade["adjclose"]) / trade["adjclose"]
                                    trade["hpr"] = sell_trade["hpr"]
                                    trade["industry"] = industry
                                    blacklist = blacklist.append([{"ticker":trade["ticker"],"start":date,"end":trade["sell_date"]}])
                                    trades.append(trade)
                                    date = trade["sell_date"] + timedelta(days=1)
                                    break
                            except Exception as e:
                                print(date,str(e))
                                date = date + timedelta(days=1)
                                continue
                        except Exception as e:
                            print(date,str(e))
                            date = date + timedelta(days=1)
                            continue
                else:
                    date = date + timedelta(days=1)
                    continue
        return trades

In [ ]:
epoch = 0
seats = list(sp5["GICS Sector"].unique())
strat_db.connect()
strat_db.drop_table("headless_epochs")
delta_range = range(3,12,3)
delta_columns = ["100d","7d"]
value_settings = [True,False]
hpr_settings = [7,14]
iterations = len(hpr_settings) * len(value_settings) * len(delta_columns) * len(delta_range)
print(iterations)
for i in range(iterations):
    strat_db.drop_table("headless_{}".format(i))
for value in tqdm(value_settings):
    for hpr in tqdm(hpr_settings):
        for delta_column in tqdm(delta_columns):
            for delta in tqdm(delta_range):
                dr = delta/100
                epoch_dict = {"epoch":epoch
                              ,"dataset":dataset
                              ,"dr":dr
                              ,"value_setting":value
                              ,"hpr":hpr
                              ,"delta_column":delta_column
                             }
                ts = backtest(sim.copy(),seats,delta_column,value,hpr,dr)
                if len(ts) > 0:
                    strat_db.store_data("headless_epochs",pd.DataFrame([epoch_dict]))
                    strat_db.store_data("headless_{}".format(epoch),pd.DataFrame(ts))
                epoch += 1
strat_db.close()

  0%|                                                                                                                           | 0/2 [00:00<?, ?it/s]

  0%|                                                                                                                           | 0/2 [00:00<?, ?it/s]


  0%|                                                                                                                           | 0/3 [00:00<?, ?it/s]



  0%|                                                                                                                          | 0/11 [00:00<?, ?it/s]

24






  9%|██████████▎                                                                                                       | 1/11 [01:27<14:32, 87.28s/it]



 18%|████████████████████▋                                                                                             | 2/11 [02:50<12:53, 85.93s/it]



 27%|███████████████████████████████                                                                                   | 3/11 [04:23<11:46, 88.31s/it]



 36%|█████████████████████████████████████████▍                                                                        | 4/11 [05:59<10:32, 90.36s/it]



 45%|███████████████████████████████████████████████████▊                                                              | 5/11 [07:28<09:00, 90.09s/it]



 55%|██████████████████████████████████████████████████████████████▏                                                   | 6/11 [09:10<07:48, 93.61s/it]



 64%|███████████████████████████████████████████████████████████████████

 82%|████████████████████████████████████████████████████████████████████████████████████████████▍                    | 9/11 [19:08<04:19, 129.84s/it]



 91%|█████████████████████████████████████████████████████████████████████████████████████████████████████▊          | 10/11 [21:14<02:08, 128.58s/it]



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [23:39<00:00, 129.08s/it]



 33%|█████████████████████████████████████▋                                                                           | 1/3 [23:40<47:20, 1420.32s/it]



  0%|                                                                                                                          | 0/11 [00:00<?, ?it/s]



  9%|██████████▎                                                                                                      | 1/11 [01:52<18:49, 112.90s/it]



 18%|████████████████████▌                                                  

 18%|████████████████████▋                                                                                             | 2/11 [01:43<07:49, 52.16s/it]



 27%|███████████████████████████████                                                                                   | 3/11 [02:39<07:07, 53.39s/it]



 36%|█████████████████████████████████████████▍                                                                        | 4/11 [03:40<06:29, 55.69s/it]



 45%|███████████████████████████████████████████████████▊                                                              | 5/11 [04:38<05:38, 56.38s/it]



 55%|██████████████████████████████████████████████████████████████▏                                                   | 6/11 [05:48<05:02, 60.45s/it]



 64%|████████████████████████████████████████████████████████████████████████▌                                         | 7/11 [06:42<03:53, 58.33s/it]



 73%|███████████████████████████████████████████████████████████████████████

...





  9%|██████████▎                                                                                                       | 1/11 [01:14<12:26, 74.67s/it]



 18%|████████████████████▋                                                                                             | 2/11 [02:32<11:19, 75.49s/it]



 27%|███████████████████████████████                                                                                   | 3/11 [03:51<10:14, 76.78s/it]



 36%|█████████████████████████████████████████▍                                                                        | 4/11 [05:20<09:23, 80.46s/it]



 45%|███████████████████████████████████████████████████▊                                                              | 5/11 [06:49<08:17, 82.88s/it]



 55%|██████████████████████████████████████████████████████████████▏                                                   | 6/11 [08:16<07:00, 84.16s/it]



 64%|███████████████████████████████████████████████████████████████████

not value






  9%|██████████▎                                                                                                       | 1/11 [01:12<12:07, 72.73s/it]



 18%|████████████████████▋                                                                                             | 2/11 [02:27<11:00, 73.36s/it]



 27%|███████████████████████████████                                                                                   | 3/11 [03:44<09:56, 74.53s/it]



 36%|█████████████████████████████████████████▍                                                                        | 4/11 [05:08<08:59, 77.13s/it]



 45%|███████████████████████████████████████████████████▊                                                              | 5/11 [06:29<07:49, 78.32s/it]



 55%|██████████████████████████████████████████████████████████████▏                                                   | 6/11 [07:52<06:39, 79.83s/it]



 64%|███████████████████████████████████████████████████████████████████

not value






  9%|██████████▎                                                                                                      | 1/11 [02:01<20:11, 121.17s/it]



 18%|████████████████████▌                                                                                            | 2/11 [04:09<18:29, 123.24s/it]



 27%|██████████████████████████████▊                                                                                  | 3/11 [06:11<16:24, 123.08s/it]



 36%|█████████████████████████████████████████                                                                        | 4/11 [08:21<14:34, 124.93s/it]



 45%|███████████████████████████████████████████████████▎                                                             | 5/11 [10:22<12:23, 123.91s/it]



 55%|█████████████████████████████████████████████████████████████▋                                                   | 6/11 [12:30<10:25, 125.03s/it]